### Setup
- To use tsne on GPU
- Wandb tool

In [ ]:
# download and unpack tsnecuda from anaconda.org
!wget https://anaconda.org/CannyLab/tsnecuda/2.1.0/download/linux-64/tsnecuda-2.1.0-cuda100.tar.bz2
!tar xvjf tsnecuda-2.1.0-cuda100.tar.bz2
!cp -r site-packages/* /usr/local/lib/python3.6/dist-packages/

# create a symbolic link between the downloaded libfaiss.so file and the location python's looking at

!echo $LD_LIBRARY_PATH 
# this is probably /usr/lib64-nvidia

!ln -s /content/lib/libfaiss.so $LD_LIBRARY_PATH/libfaiss.so

--2020-08-16 03:48:06--  https://anaconda.org/CannyLab/tsnecuda/2.1.0/download/linux-64/tsnecuda-2.1.0-cuda100.tar.bz2
Resolving anaconda.org (anaconda.org)... 104.17.93.24, 104.17.92.24, 2606:4700::6811:5c18, ...
Connecting to anaconda.org (anaconda.org)|104.17.93.24|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://binstar-cio-packages-prod.s3.amazonaws.com/5d019c23c7424a015486440c/5d790a95778a870c42137059?response-content-disposition=attachment%3B%20filename%3D%22tsnecuda-2.1.0-cuda100.tar.bz2%22%3B%20filename%2A%3DUTF-8%27%27tsnecuda-2.1.0-cuda100.tar.bz2&response-content-type=application%2Fx-tar&X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Expires=60&X-Amz-Date=20200816T034806Z&X-Amz-SignedHeaders=host&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEPP%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJHMEUCIQDWpSC1jMstnhrRrz5ds91zWD131RNkFK7R%2BH5129g5IQIgfAJ1OliH4RlskcnXcfbalKGAO2dipolITgqUddXDBsYqvQMIzP%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FARAAGgw0NTU4NjQwOTgzNzgiDFRbL

#### Setup wandb tool

In [ ]:
!pip install wandb -q

     |████████████████████████████████| 1.4MB 2.8MB/s 
     |████████████████████████████████| 102kB 7.3MB/s 
     |████████████████████████████████| 122kB 14.7MB/s 
     |████████████████████████████████| 102kB 6.8MB/s 
     |████████████████████████████████| 163kB 16.3MB/s 
     |████████████████████████████████| 71kB 5.4MB/s 
     |████████████████████████████████| 71kB 5.4MB/s 


In [ ]:
import wandb
# WandB – Login to your wandb account so you can log all your metrics
wandb.login()

True

In [ ]:
wandb.init(project="aae-mnist-tf")

W&B Run: https://app.wandb.ai/baohq/aae-mnist-tf/runs/24x45wgc

### Code

In [ ]:
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from tensorflow.keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D, Reshape
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import losses
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
import keras.backend as K

import matplotlib.pyplot as plt

import numpy as np

#### Adversarial autoencoder

In [ ]:
class AdversarialAutoencoder():
    def __init__(self, img_shape=(28,28), latent_dim=8, dropout=0.0):
        self.latent_dim = latent_dim
        self.ae_optim = Adam(0.001)
        self.d_optim = Adam(0.001)
        self.img_shape = img_shape
        self.dropout = dropout
        self.initialize_models(self.img_shape, self.latent_dim)
        
    def initialize_models(self, img_shape, latent_dim):
        self.encoder = self.build_encoder(img_shape, latent_dim)
        self.decoder = self.build_decoder(latent_dim, img_shape)
        self.discriminator = self.build_discriminator(latent_dim)
        
        img = Input(shape=img_shape)
        latent = self.encoder(img)
        res_img = self.decoder(latent)
        
        self.autoencoder = Model(img, res_img)
        discri_out = self.discriminator(latent)
        
        self.generator = Model(img, discri_out)
        self.discriminator.compile(optimizer=self.d_optim,
                                    loss='binary_crossentropy')
        self.autoencoder.compile(optimizer=self.ae_optim,
                                    loss='mse')
        # generator_discriminator ~ only train generator
        for layer in self.discriminator.layers:
            layer.trainable = False
        self.generator.compile(optimizer=self.d_optim,
                                    loss='binary_crossentropy')
        
    
    def build_encoder(self, img_shape, latent_dim):
        encoder = Sequential()
        encoder.add(Flatten(input_shape=img_shape))
        encoder.add(Dense(1000, activation='relu'))
        encoder.add(Dropout(self.dropout))
        encoder.add(Dense(1000, activation='relu'))
        encoder.add(Dropout(self.dropout))
        encoder.add(Dense(latent_dim))
        
        encoder.summary()
        return encoder
    
    def build_decoder(self, latent_dim, img_shape):
        decoder = Sequential()
        decoder.add(Dense(1000, input_dim=latent_dim, activation='relu'))
        decoder.add(Dropout(self.dropout))
        decoder.add(Dense(1000, activation='relu'))
        decoder.add(Dropout(self.dropout))
        decoder.add(Dense(np.prod(img_shape), activation='sigmoid'))
        decoder.add(Reshape(img_shape))
        
        decoder.summary()
        return decoder
    
    def build_discriminator(self, latent_dim):
        discriminator = Sequential()
        discriminator.add(Dense(1000, input_dim=latent_dim, activation='relu'))
        discriminator.add(Dropout(self.dropout))
        discriminator.add(Dense(1000, activation='relu'))
        discriminator.add(Dropout(self.dropout))
        discriminator.add(Dense(1, activation='sigmoid'))
        
        discriminator.summary()
        return discriminator
    
    def samples(self, num_images=100):
        latents = 5 * np.random.uniform(-1, 1, size=(num_images, self.latent_dim))
        imgs = self.decoder.predict(latents)
        
        return imgs
    

#### Utilities functions

In [ ]:
import os
import io
from PIL import Image
from sklearn.decomposition import PCA
# from sklearn.manifold import TSNE
# from cuml.manifold import TSNE
from tsnecuda import TSNE
import seaborn as sns
import pandas as pd

def make_image_grid(imgs, shape=None, prefix=None, save_path=None, is_show=False):
    fig = plt.figure(figsize=[20,20])
    for index, img in enumerate(imgs):
        img = img.reshape(shape) if shape != img.shape else img
        ax = fig.add_subplot(10, 10, index + 1)
        ax.set_axis_off()
        ax.imshow(img, cmap='gray')
    if prefix != None and save_path != None:
        fig.savefig(os.path.join(save_path, prefix + '.png'))
        img = Image.open(os.path.join(save_path, prefix + '.png'))
    else:
        img = imgs
    if is_show:
        plt.show()
    plt.close(fig)

    return img

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
def visualize_latent_space(x, labels, n_clusters, range_lim=(-80, 80), perplexity=40, is_save=False, save_path=None):
    # tsne = TSNE(n_components=2, verbose=0, perplexity=perplexity, n_iter=1000, init='random')
    tsne = TSNE(n_components=2)
    tsne_results = tsne.fit_transform(x)
    df_subset = pd.DataFrame()
    
    df_subset['tsne-2d-one'] = tsne_results[:,0]
    df_subset['tsne-2d-two'] = tsne_results[:,1]
    df_subset['Y'] =  labels
    
    n_comps = len(np.unique(labels).tolist())
    
    plt.figure(figsize=(16,10))
    sns_plot = sns.scatterplot(
        x='tsne-2d-one', y='tsne-2d-two',
        hue='Y',
        palette=sns.color_palette(n_colors=n_comps),
        data=df_subset,
        legend="full",
        alpha=0.3
    ).set(xlim=range_lim,ylim=range_lim)
    
    if is_save:
        if not os.path.exists(os.path.dirname(save_path)):
            os.makedirs(os.path.dirname(save_path))
        save_path = save_path if save_path else ''
        plt.savefig(save_path)
        plt.close('all')
        img = Image.open(save_path)
        return img
    return None

In [ ]:
from collections import defaultdict
def avg_losses(total_losses):
    flatten_losses = defaultdict(list)
    for loss in total_losses:
        for kv in loss.items():
            flatten_losses[kv[0]].append(kv[1])

    avg_loss = {kv[0]: sum(kv[1]) / len(kv[1]) for kv in flatten_losses.items()}

    return avg_loss

#### Training procedure

In [ ]:
import tqdm
def train(model: AdversarialAutoencoder, x_train, y_train,
          batch_size, epochs=1000, save_interval=200,
          save_path='./images'):
    # latents = 5 * np.random.normal(size=(100, model.latent_dim))
    latents = 5 * np.random.uniform(-1, 1, size=(100, model.latent_dim))
    n_epochs = tqdm.tqdm_notebook(range(epochs))
    half_batch = int(batch_size / 2)
    total_batches = x_train.shape[0] // batch_size
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    for epoch in n_epochs:
        losses = []
        offset = 0
        for batch_iter in range(total_batches):
            # Randomly choose each half batch
            imgs = x_train[offset:offset + batch_size,::] if (batch_iter < (total_batches - 1)) else x_train[offset:,::]
            offset += batch_size
            # Randomly choose each half batch
            idx = np.random.randint(0, x_train.shape[0], half_batch)
            imgs = x_train[idx]
            
            # Train discriminator
            ## Get fake and real latent feature
            latent_fake = model.encoder.predict(imgs)
            latent_real = 5 * np.random.normal(size=(half_batch, model.latent_dim))
            d_real = np.ones((half_batch, 1))
            d_fake = np.zeros((half_batch, 1))
            
            ## train
            d_loss_real = model.discriminator.train_on_batch(latent_real, d_real)
            d_loss_fake = model.discriminator.train_on_batch(latent_fake, d_fake)
            d_loss = d_loss_real + d_loss_fake
            
            idx = np.random.randint(0, x_train.shape[0], batch_size)
            imgs = x_train[idx]
            
            # Train autoencoder
            ae_loss = model.autoencoder.train_on_batch(imgs, imgs)
            
            # Train generator
            g_real = np.ones((batch_size, 1))
            g_loss = model.generator.train_on_batch(imgs, g_real)
        
            loss = {
                'ae_loss': ae_loss,
                'g_loss': g_loss,
                'd_loss': d_loss
            }
            losses.append(loss)

        avg_loss = avg_losses(losses)
        wandb.log({'losses': avg_loss})
        
        if epoch % save_interval == 0 or (epoch == epochs - 1):
            sampled_imgs = model.decoder(latents, training=False)
            res_img = make_image_grid(sampled_imgs.numpy(), (28,28), str(epoch), save_path)
            
            latent = model.encoder.predict(x_train)
            latent_space_img = visualize_latent_space(latent, y_train, 10, is_save=True, save_path=f'./latent_space/{epoch}.png')
            wandb.log({'samples_imgs': [wandb.Image(res_img, caption="Sampled images")],
                        'latent_space': [wandb.Image(latent_space_img, caption="Latent space")]})

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype(np.float32) / 255.
x_test = x_test.astype(np.float32) / 255.
ann = AdversarialAutoencoder(dropout=0.5)
train(model=ann,
          x_train=x_train,
          y_train=y_train,
          batch_size=x_train.shape[0]//4,
          epochs=2000,
          save_interval=50,
          save_path='./images')
# generated = ann.generateImages(10000)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 1000)              785000    
_________________________________________________________________
dropout (Dropout)            (None, 1000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 8008      
Total params: 1,794,008
Trainable params: 1,794,008
Non-trainable params: 0
______________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


In [ ]:
# ann.autoencoder.save('acai/autoencoder')
ann.discriminator.save('/content/wandb/run-20200816_034843-24x45wgc/aae/discriminator')
ann.encoder.save('/content/wandb/run-20200816_034843-24x45wgc/aae/encoder')
ann.decoder.save('/content/wandb/run-20200816_034843-24x45wgc/aae/decoder')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/wandb/run-20200816_034843-24x45wgc/aae/discriminator/assets
INFO:tensorflow:Assets written to: /content/wandb/run-20200816_034843-24x45wgc/aae/encoder/assets
INFO:tensorflow:Assets written to: /content/wandb/run-20200816_034843-24x45wgc/aae/decoder/assets


In [ ]:
!zip /content/wandb/run-20200816_034843-24x45wgc/aae.zip -r /content/wandb/run-20200816_034843-24x45wgc/aae/

  adding: content/wandb/run-20200816_034843-24x45wgc/aae/ (stored 0%)
  adding: content/wandb/run-20200816_034843-24x45wgc/aae/decoder/ (stored 0%)
  adding: content/wandb/run-20200816_034843-24x45wgc/aae/decoder/variables/ (stored 0%)
  adding: content/wandb/run-20200816_034843-24x45wgc/aae/decoder/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: content/wandb/run-20200816_034843-24x45wgc/aae/decoder/variables/variables.index (deflated 46%)
  adding: content/wandb/run-20200816_034843-24x45wgc/aae/decoder/saved_model.pb (deflated 90%)
  adding: content/wandb/run-20200816_034843-24x45wgc/aae/decoder/assets/ (stored 0%)
  adding: content/wandb/run-20200816_034843-24x45wgc/aae/discriminator/ (stored 0%)
  adding: content/wandb/run-20200816_034843-24x45wgc/aae/discriminator/variables/ (stored 0%)
  adding: content/wandb/run-20200816_034843-24x45wgc/aae/discriminator/variables/variables.data-00000-of-00001 (deflated 18%)
  adding: content/wandb/run-20200816_034843-24x45wgc/aa